# TFG: Alfonso Moure

Este fichero contiene un borrador del proyecto TFG de Alfonso Moure, donde se exploran diferentes técnicas para el trabajo con redes neuronales gráficas.

## Instalando los módulos necesarios

Como primer paso, se va a proceder a instalar los módulos necesarios para las pruebas:

* [numpy](https://numpy.org/), para poder realizar las operaciones necesarias sobre los datos.
* [TensorFlow](https://www.tensorflow.org/), como librería principal para ejecutar los distintos algoritmos de aprendizaje computacional a los que será necesario recurrir.
* [Spektral](https://graphneural.network/), una cómoda librería y colección de sets de datos que permite trabajar de manera cómoda con redes neuronales gráficas.

Una vez instalados, se procede a imporarlos. Aunque se incluye la descripción del fichero requierements.txt con el proyecto, se proceden a importar desde el notebook para aportar una mayor claridad.

In [1]:
# Install required modules
!pip install numpy
!pip install tensorflow
!pip install spektral

In [2]:
# Import downloaded modules
import numpy as np
import tensorflow as tf
from tensorflow import keras # To be able to use Keras more easily from the code
import spektral

Además, se crea un fichero de configuración para Spektral en el directorio raíz del usuario, tal y como se indica en la configuración de dicho módulo. Se crea en la ruta `~/.spektral/config.json` y se guarda el siguiente contenido:

```
{
        "dataset_folder": "/Users/ghostmou/vscode-projects/uoc-tfg-gnn/spektraldata"
}
```

Esto indica a Spektral que se desea guardar la información descargada para los juegos de datos que van a ser usados en la ruta indicada.

## Carga del juego de datos para las pruebas

Como se indica en la memoria de proyecto, se hará uso del juego de datos conocido como CORA. Gracias a Spektral, es sencillo descargar este juego de datos y sus distintas estructuras ya preparadas.

In [3]:
# Download CORA dataset and its different members
dataset = spektral.datasets.citation.Citation(
    'cora', 
    random_split=False, # split randomly: 20 nodes per class for training, 30 nodes 
        # per class for validation; or "Planetoid" (Yang et al. 2016)
    normalize_x=False,  # normalize the features
    dtype=np.float32 # numpy data type for the graph data
    )
dataset.graphs[0]

# Also load a list of labels as names, justo to be able to use it
label_names = ['Case_Based', 'Genetic_Algorithms', 'Neural_Networks', 'Probabilistic_Methods', 'Reinforcement_Learning', 'Rule_Learning', 'Theory']

/Users/ghostmou/virtualenvs/uoc-tfg-gnn/lib/python3.9/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Con el conjunto de datos descargado, podemos ver que el resumen nos muestra que se ha descargado un grafo con las siguientes características:

* 2708 nodos o vértices forman el grafo.
* 1433 atributos de nodo.
* 0 atributos de relación, es decir, es un nodo cuyas aristas no contienen información.
* 7 clases. En base a la definición del juego de datos, sabemos que los vértices se clasifican en dicho número de grupos. Sin embargo, como se explica en la memoria de proyecto, también es posible hacer uso de GNNs para clasificar grafos, por lo que Spektral nos permite trabajar con dos tipos de etiquetas: de nodo y de grafo.

Por otro lado, el conjunto de datos recuperado ya viene dividido en tres grupos de muestras:

* Muestras de entrenamiento, que aparecen marcadas mediante enmascaramiento con la estructura `mask_tr`.
* Muestras de validación, que hacen lo propio mediante `mask_va`.
* Muestras de prueba o test, enmascaradas con `mask_te`.

In [4]:
print(f'Training samples: {np.sum(dataset.mask_tr)}')
print(f'Validation samples: {np.sum(dataset.mask_va)}')
print(f'Test samples: {np.sum(dataset.mask_te)}')

Training samples: 140
Validation samples: 500
Test samples: 1000


## Preparación TensorBoard para seguimiento

Antes de proceder con la implementación de los ejemplos, se prepara un entorno basado en TensorBoard para poder visualizar y analizar la ejecución  y sus resultados de manera visual.

In [5]:
# Import TensorBoard callback to be able to use it in all the code samples
from keras.callbacks import TensorBoard

# Prepare placement for the logs
import os
root_logdir = os.path.join(os.curdir, 'my_logs')

def get_run_logdir(model_in_use):
    import time
    run_id = time.strftime(f'run_{model_in_use}_%Y_%m_%d-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

## Implementación de pruebas

A continuación, se va a trabajar en la implementación de varios modelos diferentes de ConvGNN para explicar su funcionamiento:

* ConvGNN espectral, con un ejemplo de clasificación de nodos.
* ConvGNN espacial con paso de mensajes o MPNN, con un ejemplo de clasificación de nodos.

Cada apartado irá rematado con un estudio de precisión. Todos los casos serán ejecutados mediante GridSearch para intentar encontrar el resultado más óptimo bajo las condiciones actuales. Además, se extraerá información gráfica mediante TensorBoard.

### Implementación 1: ConvGNN espectral




Puede verse que se cuenta con 140 muestras de entrenamiento, 500 de validación y 1000 de test.

Dado que este enmascaramiento se lleva a cabo mediante datos binarios (verdadero o falso; incluido o excluido de cada subconjunto, respectivamente), es preciso transformar estos valores en pesos que puedan ser usados durante el proceso de aprendizaje: sabemos que las CNN espestrales no hacen uso de los features de nodos y vértices para su toma de decisiones.

Para ello, se crea una función que convierte estas colecciones en una media de peso de los valores de los nodos.

**ESTE BLOQUE REQUIERE REVISIÓN DESDE MIS NOTAS**

In [6]:
weighed_mask = [
    mask.astype(np.float32) / np.count_nonzero(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
]

Así, cada vértice de cada colección de muestras tendrá asignado un peso igual al del resto de su conjunto:

In [7]:
print(f'Training samples weight: {np.nanmean(np.where(weighed_mask[0] > 0, weighed_mask[0],np.nan), 0)}')
print(f'Validation samples weight: {np.nanmean(np.where(weighed_mask[1] > 0, weighed_mask[1],np.nan), 0)}')
print(f'Test samples weight: {np.nanmean(np.where(weighed_mask[2] > 0, weighed_mask[2],np.nan), 0)}')

Training samples weight: 0.0071428571827709675
Validation samples weight: 0.0020000000949949026
Test samples weight: 0.0010000001639127731


Preparados los datos, se puede proceder a definir el modelo. Para ello, se hará uso del existente GCN procedente de Spektral (`spektral.models.gcn`).

In [8]:
# Import model
from spektral.models.gcn import GCN

# Import optimizers
from tensorflow.keras.optimizers import Adam

# Import loss function from Keras
from tensorflow.keras.losses import CategoricalCrossentropy

# Set initial configuration
learning_rate = 0.01
reduction_function = 'sum'

# Create model from Spektral
model_gcn = GCN(n_labels=dataset.n_labels, n_input_channels=dataset.n_node_features)

# Compile loaded model
model_gcn.compile(
    optimizer=Adam(learning_rate=learning_rate), # Set optimizer as Adam with a learning rate of 0.01
    loss=CategoricalCrossentropy(reduction=reduction_function), # Loss function to be used.
    weighted_metrics=['acc'] # Metrics to be evaluated and weighted during training (Keras doc: https://keras.io/api/models/model_training_apis/)
)

2021-12-05 19:45:44.339440: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ahora que tenemos el modelo creado y compilado, es posible proceder con su entrenamiento. Para ello, se empieza por crear los cargadores (*Loaders*) de Spektral para entrenamiento y validación, básicos para las tareas de entrenamiento. Dentro de Spektral, los *loaders* son usados para generar lotes de subgrafos para poder hacer sucesivas pasadas de entrenamiento en la red convolucional en uso. 

Gracias a que Spektral está diseñado para trabajar con Keras, la clase `Loader` incluye un método `load` que puede ser usado en para llamar al método `fit` del modelo.

Puesto que el conjunto de datos incluye un solo grafo, es posible hacer uso del cargador `SingleLoader`, diseñado para trabajar con este tipo de estructuras de grafo único. Puede ser configurado mediante los siguientes parámetros pasados durante su inicialización:

* `dataset`, que incluye, como puede deducirse, la estructura de datos.
* `epochs`, con la cantidad de **epochs** que van a ser usados durante la fase de entrenamiento.
* `shuffle`, para indicar si se desea barajar los contenidos del conjunto de datos tras cada epoch.
* `sample_weights`, que podrá ser usado para indicar el peso de cada observación y que, en caso de ser usado en la inicialización, será devuelto en cada paso de entrenamiento. Esta es la estructura que hemos generado con anterioridad para cada subconjunto de entrenamiento, pruebas y validación en base a las estructuras binarias originales. Así, se usará un vector de pesos diferente según el cargador que estemos preparando.

Puesto que el siguiente paso es llevar a cabo el entrenamiento del modelo, se prepararán los cargadores de los datos de entrenamiento y validación mediante `SingleLoader`.

In [9]:
from spektral.data.loaders import SingleLoader
loader_training = SingleLoader(dataset, sample_weights=weighed_mask[0])
loader_validation = SingleLoader(dataset, sample_weights=weighed_mask[1])

Con los cargadores listos, es posible proceder a entrenar el modelo mediante la función `fit`. A modo de prueba, se hará con un total de `50` epochs. Además, se incorpora un callback de Keras, `EarlyStopping`, que permitirá detener el entrenamiento cuando el proceso detecte que no se está obteniendo una mejora sustancial.

Como se mencionó más arriba, se incorpora también un callback `TensorBoard` para poder analizar las pruebas. Sus resultados serán accesibles mediante el siguiente comando de consola:

```shell
tensorboard --logdir=./my_logs --port=6006
```

In [10]:
# Fit the model
from tensorflow.keras.callbacks import EarlyStopping
model_gcn.fit(
    loader_training.load(),
    steps_per_epoch=loader_training.steps_per_epoch,
    validation_data=loader_validation.load(),
    validation_steps=loader_validation.steps_per_epoch,
    epochs=50,
    callbacks=[
        EarlyStopping(patience=10,  restore_best_weights=True), # Early stopping callback
        TensorBoard(get_run_logdir('gcn_spectral'))
    ]
)

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 5.9384 - acc: 0.0786 - val_loss: 2.2536 - val_acc: 0.2820
Epoch 2/50
1/1 [==============================] - 0s 29ms/step - loss: 2.8648 - acc: 0.2786 - val_loss: 1.7124 - val_acc: 0.3200
Epoch 3/50
1/1 [==============================] - 0s 32ms/step - loss: 1.6937 - acc: 0.3000 - val_loss: 1.5578 - val_acc: 0.4300
Epoch 4/50
1/1 [==============================] - 0s 31ms/step - loss: 1.5336 - acc: 0.5000 - val_loss: 1.5080 - val_acc: 0.4900
Epoch 5/50
1/1 [==============================] - 0s 28ms/step - loss: 1.2681 - acc: 0.4786 - val_loss: 1.5888 - val_acc: 0.4680
Epoch 6/50
1/1 [==============================] - 0s 30ms/step - loss: 1.5334 - acc: 0.4500 - val_loss: 1.5364 - val_acc: 0.4940
Epoch 7/50
1/1 [==============================] - 0s 29ms/step - loss: 1.2792 - acc: 0.5429 - val_loss: 1.5012 - val_acc: 0.5160
Epoch 8/50
1/1 [==============================] - 0s 33ms/step - loss: 1.1374 - acc: 0.5929 - val_l

Una vez hecho este entrenamiento, se puede proceder a evaluar su eficacia.

In [11]:
loader_test = SingleLoader(dataset, sample_weights=weighed_mask[2])
results = model_gcn.evaluate(
    loader_test.load(), 
    steps=loader_test.steps_per_epoch
)
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')

1/1 [==============================] - 0s 11ms/step - loss: 1.3139 - acc: 0.6270
Loss: 1.313869833946228
Accuracy: 0.6270002722740173


### Implementación 2: ConvGNN espacial mediante paso de mensajes: MPNN

En el ejemplo anterior se ha basado el aprendizaje y la clasificación en la estructura espectral de los datos cargados, es decir, en la estructura de la matriz de adyacencia. Sin embargo, tal y como se explica en la memoria de proyecto, esta aproximación puede ser pobre para escenarios donde las relaciones (aristas) entre entidades (vértices) del grafo tienen un significado o importancia diferente, o cuando el contexto de un nodo, construido mediante los atributos de sus vecinos, es importante para la clasificación.

Para poder preparar este ejemplo, se hará uso de la clase `MessagePassing` de Spektral, que ofrece una API ya preparada para configurar la función de activación y personalizar el comportamiento para distintos juegos de datos. Así, será preciso personalizar:

* Función para la construcción del mensaje pasado entre dos vértices vía la arista que los une. Es conocida como `message` dentro de la API de Spektral.
* Selección de la función de agregación de los mensajes pasados desde cada arista a cada nodo: suma, media, etc. Aparece definida como `aggregate`.

Puesto que las GNN basadas en paso de mensajes requieren sucesivas iteraciones que permitan propagar los mensajes a niveles cada vez más lejanos, la función `propagate` lo ejecuta y computa los atributos de cada nodo tras pasar los mensajes de todas las aristas del grafo y computar la correspondiente función de agregación.

Puesto que Spektral está construido sobre Keras, es preciso, además, implementar algunos de los métodos heredados de la clase `Layer` para definir la capa gráfica espacial:

* `__init__`, donde se define la función de activación, se pasan el resto de hiperparámetros estándar y se guarda el dato del tamaño de la salida, que será usado más adelante.
* `build`, cuya misión es la de inicializar los pesos de la capa mediante la llamada a `add_weight` y asignar el tamaño que tendrá la matriz utilizada para almacenar los pesos dentro de la capa. La matriz de pesos tendrá tantas filas como la entrada y columnas como la salida.
* `call` es el método encargado de ejecutar los cálculos de la capa. Puesto que estamos hablando de paso de mensajes, la salida de la capa será función de las característica de los nodos de entrada y los pesos de la capa. Al final de la misma, en lugar de llamar a la función de activación, se llama a `propagate`, un método disponible dentro de `MessagePassing` encargado de realizar la propagación de los mensajes de cada nodo a sus vecinos.

A estos métodos estándar, se incorporan otros de la propia API de Spektral, especializados en el trabajo con redes gráficas:

* `message`, encargado de componer el mensaje que será pasado entre los nodos. En este ejemplo se hace uso de la información de los nodos vecinos, accesibles mediante `get_j`.
* `aggregate`, con el cometido de definir la función de agregación para los mensajes del vecindario. Aquí se hace uso de la media de todos los mensajes del vecindario, aunque una posible mejora del modelo podría ser el uso de hiperparámetros para encontrar la que mejores reusltados pueda dar. Esta opción es posible desde el propio constructor.
* `update` donde, ahora sí, se hace uso de la función de activaciónn apra actualizar la matriz de pesos.

Con todo, se empieza creando la clase MPNNLayer como herencia `MessagePassing` para preparar la personalización de los citados métodos y definir la capa de paso de mensajes del modelo a usar.

In [12]:
from spektral.layers import MessagePassing

class MPNNLayer(MessagePassing):
    def __init__(self, n_out, activation, **kwargs):
        # Initialize message passing layer with the activation function chosen when creating the model
        super().__init__(activation=activation, **kwargs)
        self.n_out = n_out

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            shape=(batch_input_shape[0][-1], self.n_out)
        )

    def call(self, inputs):
        x, a = inputs

        # Update node features based on inputs by multiplying node attributes by 
        # the weights stored during build.
        # Kipf 2016
        x = tf.matmul(x, self.kernel)

        # Return propagation result
        return self.propagate(x=x, a=a)

    def message(self, x):
        return self.get_j(x)
    
    def aggregate(self, messages): 
        # We need to return the result of applying the aggregate function over the messages

        # Try to use the mean as aggregate function. We use a scatter mean method for this experiment:
        return spektral.layers.ops.scatter_mean(messages, self.index_i, self.n_nodes)

    def update(self, embeddings):
        return self.activation(embeddings)


Creada la capa, podemos hacer uso de la misma creando un modelo mediante la API funcional de Keras. Para ello, se empieza por configurar algunos parámetros básicos para su funcionamiento:

* Ratio de regularización (o *regularization rate*): con el fin de regularizar los contenidos. Se inicia con un valor de `5e-6`.
* Ratio de aprendizaje (o *learning rate*) con un valor de 0.2.
* Número de epochs de entrenamiento, que son fijadas en 20.
* Nivel de paciencia para el early stopping que será usado más abajo en la etapa de entrenamiento.

Hecho esto, se extran las características principales de los datos: número de nodos, número de careacterísticas por nodo y número de clases para clasificarlos. Estos datos serán usados para instanciar la capa de paso de mensajes.

In [13]:
# Initial setup for the model
l2_regularization_rate = 5e-6
learning_rate = 0.2
epochs = 20
patience = 10

# Input parameters
number_of_nodes = dataset.n_nodes
number_of_node_features = dataset.n_node_features
number_of_labels = dataset.n_labels

Ahora es el momento de definir las entradas. Para ello, se crearán dos tensores:

* `adjacency_matrix` con la matriz de adyacencia, que contendrá la matriz de adyacencia y que permitirá saber las conexiones de cada nodo.
* `x_input` con las características de cada nodo.

Ambas son usadas para definir la capa de salida del modelo, que será construida mediante la clase definida para el MPNNLayer.

In [14]:

# Define input tensors
# We define two input tensors: one for the nodes and one for the adjacency matrix
from keras.layers import Input
adjacency_input = Input(
    (number_of_nodes,), sparse=True, dtype=dataset[0].a.dtype,
    name='adjacency_matrix_input'
)
x_input = Input(
    shape=(number_of_node_features,),
    name='nodes_input'
)

# Define output layer based on the MPNN layer defined previously as MPNNLayer
mpnn_output_layer = MPNNLayer(
    number_of_labels, activation=keras.activations.softmax,
    kernel_regularizer=keras.regularizers.l2(l2_regularization_rate),
    use_bias=False, name='mpnn_layer'
)([x_input, adjacency_input])

Mediante una herencia sonbre `keras.models.Model` se puede definir un modelo que tendrá:

* Tendrá dos entradas formadas por las dos variables definidas con anterioridad: los datos de cada nodo en `x_input` y la matriz de adyacencia que los relaciona entre sí en `adjacency_matrix`.
* La salida será la capa creada en el paso anterior, que ha sido denominada como `mpnn_output`.

In [15]:

# Build and compile model
from keras.models import Model
model_mpnn = Model(
    inputs=[x_input, adjacency_input],
    outputs=mpnn_output_layer
)
model_mpnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
    loss=keras.losses.CategoricalCrossentropy(),
    weighted_metrics=['acc']
)
model_mpnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 nodes_input (InputLayer)       [(None, 1433)]       0           []                               
                                                                                                  
 adjacency_matrix_input (InputL  [(None, 2708)]      0           []                               
 ayer)                                                                                            
                                                                                                  
 mpnn_layer (MPNNLayer)         (None, 7)            10031       ['nodes_input[0][0]',            
                                                                  'adjacency_matrix_input[0][0]'] 
                                                                                              

En el resumen impreso del modelo pueden verse las dos capas de entrada descritas, que sirven como entrada para la capa de paso de mensajes. A ellas se conecta la capa de paso de mensajes.

Construido el modelo, puede ser entrenado.

In [16]:
# Train the model
loader_tr = SingleLoader(dataset, sample_weights=dataset.mask_tr)
loader_va = SingleLoader(dataset, sample_weights=dataset.mask_va)
model_mpnn.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[
        EarlyStopping(patience=patience, restore_best_weights=True),
        TensorBoard(get_run_logdir('gcn_spatial'))
    ],
)

Epoch 1/20
1/1 [==============================] - 1s 575ms/step - loss: 0.1003 - acc: 0.1786 - val_loss: 0.2189 - val_acc: 0.6420
Epoch 2/20
1/1 [==============================] - 0s 18ms/step - loss: 0.0149 - acc: 0.9857 - val_loss: 0.1651 - val_acc: 0.7020
Epoch 3/20
1/1 [==============================] - 0s 19ms/step - loss: 0.0031 - acc: 0.9929 - val_loss: 0.1625 - val_acc: 0.7180
Epoch 4/20
1/1 [==============================] - 0s 18ms/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.1714 - val_acc: 0.7180
Epoch 5/20
1/1 [==============================] - 0s 17ms/step - loss: 4.4914e-04 - acc: 1.0000 - val_loss: 0.1829 - val_acc: 0.7140
Epoch 6/20
1/1 [==============================] - 0s 16ms/step - loss: 2.2372e-04 - acc: 1.0000 - val_loss: 0.1948 - val_acc: 0.7200
Epoch 7/20
1/1 [==============================] - 0s 17ms/step - loss: 1.2248e-04 - acc: 1.0000 - val_loss: 0.2062 - val_acc: 0.7160
Epoch 8/20
1/1 [==============================] - 0s 20ms/step - loss: 7.1937e-05 - 

Entrenado, podemos evaluar el modelo frente al conjunto de test.

In [17]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=dataset.mask_te)
eval_results = model_mpnn.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 13ms/step - loss: 0.3273 - acc: 0.7320
Done.
Test loss: 0.3273445665836334
Test accuracy: 0.7319999933242798


### Implementación 3: clasificación de nodos mediante CNN sin uso de estructura gráfica

El primer paso será construir una estructura de datos que nos permita trabajar con un modelo que no esté diseñado para operar sobre grafos. Para ello, se extraerán las características de cada observación de los datos de origen y no se usarán sus enlaces. Con todo, el objetivo es medir el nivel de precisión a la hora de clasificar sin utilizar la estructura generado mediante las relaciones entre nodos.

In [18]:
# Extract train, validation and test features and labels
X_train = dataset[0].x[np.array(dataset.mask_tr)]
y_train = dataset[0].y[np.array(dataset.mask_tr)]
X_validation = dataset[0].x[np.array(dataset.mask_va)]
y_validation = dataset[0].y[np.array(dataset.mask_va)]
X_test = dataset[0].x[np.array(dataset.mask_te)]
y_test = dataset[0].y[np.array(dataset.mask_te)]

print(f'Training: X {X_train.shape}, y {y_train.shape}, from a source of {np.sum(dataset.mask_tr)} samples')
print(f'Validation: X {X_validation.shape}, y {y_validation.shape}, from a source of {np.sum(dataset.mask_va)} samples')
print(f'Test: X {X_test.shape}, y {y_test.shape}, from a source of {np.sum(dataset.mask_te)} samples')


Training: X (140, 1433), y (140, 7), from a source of 140 samples
Validation: X (500, 1433), y (500, 7), from a source of 500 samples
Test: X (1000, 1433), y (1000, 7), from a source of 1000 samples


In [19]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=X_train[0].shape),
    keras.layers.Dense(300, activation=keras.activations.relu),
    keras.layers.Dense(7, activation=keras.activations.relu)
])

# Compile model (based on the same parameters used with the espectral ConvGNN)
model.compile(
    optimizer=Adam(learning_rate=0.01), # Set optimizer as Adam with a learning rate of 0.01
    loss=CategoricalCrossentropy(reduction=reduction_function), # Loss function to be used.
    weighted_metrics=['acc'] # Metrics to be evaluated and weighted during training (Keras doc: https://keras.io/api/models/model_training_apis/)
)

model.fit(
    X_train, y_train, epochs=30, 
    validation_data=(X_validation, y_validation),
    callbacks=[
        EarlyStopping(patience=10,  restore_best_weights=True), # Early stopping callback
        TensorBoard(get_run_logdir('non_gnn'))
    ]
)

Epoch 1/30
5/5 [==============================] - 0s 35ms/step - loss: 231.7274 - acc: 0.3143 - val_loss: 248.2600 - val_acc: 0.3580
Epoch 2/30
5/5 [==============================] - 0s 12ms/step - loss: 148.4955 - acc: 0.6357 - val_loss: 254.9398 - val_acc: 0.3600
Epoch 3/30
5/5 [==============================] - 0s 12ms/step - loss: 144.1733 - acc: 0.7071 - val_loss: nan - val_acc: 0.3640
Epoch 4/30
5/5 [==============================] - 0s 13ms/step - loss: nan - acc: 0.6500 - val_loss: nan - val_acc: 0.1220
Epoch 5/30
5/5 [==============================] - 0s 13ms/step - loss: nan - acc: 0.1429 - val_loss: nan - val_acc: 0.1220
Epoch 6/30
5/5 [==============================] - 0s 14ms/step - loss: nan - acc: 0.1429 - val_loss: nan - val_acc: 0.1220
Epoch 7/30
5/5 [==============================] - 0s 14ms/step - loss: nan - acc: 0.1429 - val_loss: nan - val_acc: 0.1220
Epoch 8/30
5/5 [==============================] - 0s 12ms/step - loss: nan - acc: 0.1429 - val_loss: nan - val_acc

In [20]:
results = model.evaluate(X_test, y_test)
print(f'Loss: {results[0]}')
print(f'Accuracy: {results[1]}')

32/32 [==============================] - 0s 1ms/step - loss: 239.7379 - acc: 0.3700
Loss: 239.73793029785156
Accuracy: 0.3700000047683716


### Implementación 4: predicción de nodos mediante paso de mensajes

En este caso, la implementación será hecha mediante el modelo WalkPooling e implementado mediante la librería Pytorch, según el planteamiento en https://paperswithcode.com/sota/link-prediction-on-cora

#TODO Añadir https://paperswithcode.com/sota/link-prediction-on-cora a bibliografía

In [21]:
# Clone repository from Walkpooling paper https://github.com/DaDaCheng/WalkPooling/
# TODO put here the reference from the memory
#%%bash
#git clone https://github.com/DaDaCheng/WalkPooling/

In [22]:
# Install necessary modules in current environment
necessary_modules = ['torch', 'torch-cluster', 'torch-scatter', 'torch-sparse', 'torch-geometric', 'tqdm']
for module in necessary_modules:
    if module not in sys.modules:
        !pip install {module}

# %load WalkPooling/src/model.py
import torch
from torch.nn import Linear, Parameter,Embedding
import torch.nn.functional as F
from torch_scatter import scatter_mean, scatter, scatter_add, scatter_max
from torch_geometric.nn.conv import MessagePassing
import torch.nn as nn
from torch_geometric.utils import softmax
from torch_geometric.nn import GCNConv 
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class LinkPred(MessagePassing):
    def __init__(self, in_channels: int, hidden_channels: int, heads: int = 1,\
                 walk_len: int = 6, drnl: bool = False, z_max: int =100, MSE: bool=True):
        super(LinkPred, self).__init__()

        self.drnl = drnl
        if drnl == True:
            self.z_embedding = Embedding(z_max, hidden_channels)

        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)

        self.wp = WalkPooling(in_channels + hidden_channels*2,\
            hidden_channels, heads, walk_len)

        L=walk_len*5+1
        self.classifier = MLP(L*heads,MSE=MSE)


    def forward(self, x, edge_index, edge_mask, batch, z = None):
        
        #using drnl
        if self.drnl == True:
            z_emb = self.z_embedding(z)
            if z_emb.ndim == 3:  # in case z has multiple integer labels
                z_emb = z_emb.sum(dim=1)
            z_emb = z_emb.view(x.size(0),-1)
            x = torch.cat((x,z_emb.view(x.size(0),-1)),dim=1)
        
        #GCN layers
        x_out = x
        x = self.conv1(x, edge_index)
        x_out = torch.cat((x_out,x),dim=1)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x_out = torch.cat((x_out,x),dim=1)

        #Walk Pooling
        feature_list = self.wp(x_out, edge_index, edge_mask, batch)

        #Classifier
        out = self.classifier(feature_list)

        return out


class WalkPooling(MessagePassing):
    def __init__(self, in_channels: int, hidden_channels: int, heads: int = 1,\
                 walk_len: int = 6):
        super(WalkPooling, self).__init__()

        self.hidden_channels = hidden_channels
        self.heads = heads
        self.walk_len = walk_len

        # the linear layers in the attention encoder
        self.lin_key1 = Linear(in_channels, hidden_channels)
        self.lin_query1 = Linear(in_channels, hidden_channels)
        self.lin_key2 = Linear(hidden_channels, heads * hidden_channels)
        self.lin_query2 = Linear(hidden_channels, heads * hidden_channels)
    def attention_mlp(self, x, edge_index):
    
        query = self.lin_key1(x).reshape(-1,self.hidden_channels)
        key = self.lin_query1(x).reshape(-1,self.hidden_channels)

        query = F.leaky_relu(query,0.2)
        key = F.leaky_relu(key,0.2)

        query = F.dropout(query, p=0.5, training=self.training)
        key = F.dropout(key, p=0.5, training=self.training)

        query = self.lin_key2(query).view(-1, self.heads, self.hidden_channels)
        key = self.lin_query2(key).view(-1, self.heads, self.hidden_channels)

        row, col = edge_index
        weights = (query[row] * key[col]).sum(dim=-1) / np.sqrt(self.hidden_channels)
        
        return weights

    def weight_encoder(self, x, edge_index, edge_mask):        
     
        weights = self.attention_mlp(x, edge_index)
    
        omega = torch.sigmoid(weights[torch.logical_not(edge_mask)])
        
        row, col = edge_index
        num_nodes = torch.max(edge_index)+1

        # edge weights of the plus graph
        weights_p = softmax(weights,edge_index[1])

        # edge weights of the minus graph
        weights_m = weights - scatter_max(weights, col, dim=0, dim_size=num_nodes)[0][col]
        weights_m = torch.exp(weights_m)
        weights_m = weights_m * edge_mask.view(-1,1)
        norm = scatter_add(weights_m, col, dim=0, dim_size=num_nodes)[col] + 1e-16
        weights_m = weights_m / norm

        return weights_p, weights_m, omega

    def forward(self, x, edge_index, edge_mask, batch):
        
        #encode the node representation into edge weights via attention mechanism
        weights_p, weights_m, omega = self.weight_encoder(x, edge_index, edge_mask)

        # pytorch geometric set the batch adjacency matrix to
        # be the diagonal matrix with each graph's adjacency matrix
        # stacked in the diagonal. Therefore, calculating the powers
        # of the stochastic matrix directly will cost lots of memory.
        # We compute the powers of stochastic matrix as follows
        # Let A = diag ([A_1,\cdots,A_n]) be the batch adjacency matrix,
        # we set x = [x_1,\cdots,x_n]^T be the batch feature matrix
        # for the i-th graph in the batch with n_i nodes, its feature 
        # is a n_i\times n_max matrix, where n_max is the largest number
        # of nodes for all graphs in the batch. The elements of x_i are
        # (x_i)_{x,y} = \delta_{x,y}. 

        # number of graphs in the batch
        batch_size = torch.max(batch)+1

        # for node i in the batched graph, index[i] is i's id in the graph before batch 
        index = torch.zeros(batch.size(0),1,dtype=torch.long)
        
        # numer of nodes in each graph
        _, counts = torch.unique(batch, sorted=True, return_counts=True)
        
        # maximum number of nodes for all graphs in the batch
        max_nodes = torch.max(counts)

        # set the values in index
        id_start = 0
        for i in range(batch_size):
            index[id_start:id_start+counts[i]] = torch.arange(0,counts[i],dtype=torch.long).view(-1,1)
            id_start = id_start+counts[i]

        index = index.to(device)
        
        #the output graph features of walk pooling
        nodelevel_p = torch.zeros(batch_size,(self.walk_len*self.heads)).to(device)
        nodelevel_m = torch.zeros(batch_size,(self.walk_len*self.heads)).to(device)
        linklevel_p = torch.zeros(batch_size,(self.walk_len*self.heads)).to(device)
        linklevel_m = torch.zeros(batch_size,(self.walk_len*self.heads)).to(device)
        graphlevel = torch.zeros(batch_size,(self.walk_len*self.heads)).to(device)
        # a link (i,j) has two directions i->j and j->i, and
        # when extract the features of the link, we usually average over
        # the two directions. indices_odd and indices_even records the
        # indices for a link in two directions
        indices_odd = torch.arange(0,omega.size(0),2).to(device)
        indices_even = torch.arange(1,omega.size(0),2).to(device)

        omega = torch.index_select(omega, 0 ,indices_even)\
        + torch.index_select(omega,0,indices_odd)
        
        #node id of the candidate (or perturbation) link
        link_ij, link_ji = edge_index[:,torch.logical_not(edge_mask)]
        node_i = link_ij[indices_odd]
        node_j = link_ij[indices_even]

        # compute the powers of stochastic matrix
        for head in range(self.heads):

            # x on the plus graph and minus graph
            x_p = torch.zeros(batch.size(0),max_nodes,dtype=x.dtype).to(device)
            x_p = x_p.scatter_(1,index,1)
            x_m = torch.zeros(batch.size(0),max_nodes,dtype=x.dtype).to(device)
            x_m = x_m.scatter_(1,index,1)

            # propagage once
            x_p = self.propagate(edge_index, x= x_p, norm = weights_p[:,head])
            x_m = self.propagate(edge_index, x= x_m, norm = weights_m[:,head])
        
            # start from tau = 2
            for i in range(self.walk_len):
                x_p = self.propagate(edge_index, x= x_p, norm = weights_p[:,head])
                x_m = self.propagate(edge_index, x= x_m, norm = weights_m[:,head])
                
                # returning probabilities around i + j
                nodelevel_p_w = x_p[node_i,index[node_i].view(-1)] + x_p[node_j,index[node_j].view(-1)]
                nodelevel_m_w = x_m[node_i,index[node_i].view(-1)] + x_m[node_j,index[node_j].view(-1)]
                nodelevel_p[:,head*self.walk_len+i] = nodelevel_p_w.view(-1)
                nodelevel_m[:,head*self.walk_len+i] = nodelevel_m_w.view(-1)
  
                # transition probabilities between i and j
                linklevel_p_w = x_p[node_i,index[node_j].view(-1)] + x_p[node_j,index[node_i].view(-1)]
                linklevel_m_w = x_m[node_i,index[node_j].view(-1)] + x_m[node_j,index[node_i].view(-1)]
                linklevel_p[:,head*self.walk_len+i] = linklevel_p_w.view(-1)
                linklevel_m[:,head*self.walk_len+i] = linklevel_m_w.view(-1)

                # graph average of returning probabilities
                diag_ele_p = torch.gather(x_p,1,index)
                diag_ele_m = torch.gather(x_m,1,index)

                graphlevel_p = scatter_add(diag_ele_p, batch, dim = 0)
                graphlevel_m = scatter_add(diag_ele_m, batch, dim = 0)

                graphlevel[:,head*self.walk_len+i] = (graphlevel_p-graphlevel_m).view(-1)
         
        feature_list = graphlevel 
        feature_list = torch.cat((feature_list,omega),dim=1)
        feature_list = torch.cat((feature_list,nodelevel_p),dim=1)
        feature_list = torch.cat((feature_list,nodelevel_m),dim=1)
        feature_list = torch.cat((feature_list,linklevel_p),dim=1)
        feature_list = torch.cat((feature_list,linklevel_m),dim=1)


        return feature_list

    def message(self, x_j, norm):
        return norm.view(-1, 1) * x_j  

class MLP(torch.nn.Module):
    # adopt a MLP as classifier for graphs
    def __init__(self,input_size,MSE=True):
        super(MLP, self).__init__()
        self.nn = nn.BatchNorm1d(input_size)
        self.linear1 = torch.nn.Linear(input_size,input_size*20)
        self.linear2 = torch.nn.Linear(input_size*20,input_size*20)
        self.linear3 = torch.nn.Linear(input_size*20,input_size*10)
        self.linear4 = torch.nn.Linear(input_size*10,input_size)
        self.linear5 = torch.nn.Linear(input_size,1)
        self.act= nn.ReLU()
        self.MSE=MSE
    def forward(self, x):
        out= self.nn(x)
        out= self.linear1(out)
        out = self.act(out)
        out= self.linear2(out)
        out = self.act(out)
        out = self.linear3(out)
        out = self.act(out)
        out = self.linear4(out)
        out = self.act(out)
        out = F.dropout(out, p=0.5, training=self.training)
        out = self.linear5(out)
        #out = torch.sigmoid(out)
        if self.MSE:
            out = torch.sigmoid(out)
        return out


In [23]:
# %load WalkPooling/software/drnl.py
"Code adopted and implemented from https://github.com/muhanzhang/SEAL"
import scipy.sparse as ssp
from scipy.sparse.csgraph import shortest_path
import torch
import numpy as np

def drnl_node_labeling(edge_index, src, dst, num_nodes):

    edge_weight = torch.ones(edge_index.size(1), dtype=int)
    adj = ssp.csr_matrix(
            (edge_weight, (edge_index[0], edge_index[1])), 
            shape=(num_nodes, num_nodes))
    # Double Radius Node Labeling (DRNL).
    src, dst = (dst, src) if src > dst else (src, dst)

    idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
    adj_wo_src = adj[idx, :][:, idx]

    idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
    adj_wo_dst = adj[idx, :][:, idx]

    dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
    dist2src = np.insert(dist2src, dst, 0, axis=0)
    dist2src = torch.from_numpy(dist2src)

    dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
    dist2dst = np.insert(dist2dst, src, 0, axis=0)
    dist2dst = torch.from_numpy(dist2dst)

    dist = dist2src + dist2dst
    dist_over_2, dist_mod_2 = dist // 2, dist % 2

    z = 1 + torch.min(dist2src, dist2dst)
    z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
    z[src] = 1.
    z[dst] = 1.
    z[torch.isnan(z)] = 0.
    return z.to(torch.int)

# def drnl_node_labeling(edge_index, src, dst, num_nodes):
#     # Distance Encoding Plus. When computing distance to src, temporarily mask dst;
#     # when computing distance to dst, temporarily mask src. Essentially the same as DRNL.
#     max_dist=100
#     edge_weight = torch.ones(edge_index.size(1), dtype=int)
#     adj = ssp.csr_matrix(
#             (edge_weight, (edge_index[0], edge_index[1])), 
#             shape=(num_nodes, num_nodes))
#     src, dst = (dst, src) if src > dst else (src, dst)

#     idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
#     adj_wo_src = adj[idx, :][:, idx]

#     idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
#     adj_wo_dst = adj[idx, :][:, idx]

#     dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True, indices=src)
#     dist2src = np.insert(dist2src, dst, 0, axis=0)
#     dist2src = torch.from_numpy(dist2src)

#     dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True, indices=dst-1)
#     dist2dst = np.insert(dist2dst, src, 0, axis=0)
#     dist2dst = torch.from_numpy(dist2dst)

#     dist = torch.cat([dist2src.view(-1, 1), dist2dst.view(-1, 1)], 1)
#     dist[dist > max_dist] = max_dist
#     dist[torch.isnan(dist)] = max_dist + 1
#     dist = torch.sum(dist,dim=1)

#     return dist.to(torch.int)


In [30]:
# %load WalkPooling/src/utils.py
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
import argparse
import numpy as np
import math
from torch_geometric.utils import to_undirected, from_scipy_sparse_matrix,dense_to_sparse,is_undirected
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import sys
import os.path
import pickle as pkl
import networkx as nx
import scipy.sparse as sp


cur_dir = os.path.abspath('') #os.path.dirname(os.path.realpath(__file__))
par_dir = os.path.abspath(os.path.join(cur_dir, '..'))#os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
#par_dir = '/Users/ghostmou/vscode-projects/uoc-tfg-gnn/WalkPooling' #os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
sys.path.append('%s/WalkPooling/software/' % cur_dir)
from drnl import drnl_node_labeling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def floor(x):
    return torch.div(x, 1, rounding_mode='trunc')

def set_random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

def split_edges(data,args):
    set_random_seed(args.seed)
    row, col = data.edge_index
    mask = row < col
    row, col = row[mask], col[mask]
    n_v= floor(args.val_ratio * row.size(0)).int() #number of validation positive edges
    n_t=floor(args.test_ratio * row.size(0)).int() #number of test positive edges
    #split positive edges   
    perm = torch.randperm(row.size(0))
    row, col = row[perm], col[perm]
    r, c = row[:n_v], col[:n_v]
    data.val_pos = torch.stack([r, c], dim=0)
    r, c = row[n_v:n_v+n_t], col[n_v:n_v+n_t]
    data.test_pos = torch.stack([r, c], dim=0)
    r, c = row[n_v+n_t:], col[n_v+n_t:]
    data.train_pos = torch.stack([r, c], dim=0)

    #sample negative edges
    if args.practical_neg_sample == False:
        # If practical_neg_sample == False, the sampled negative edges
        # in the training and validation set aware the test set

        neg_adj_mask = torch.ones(data.num_nodes, data.num_nodes, dtype=torch.uint8)
        neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
        neg_adj_mask[row, col] = 0

        # Sample all the negative edges and split into val, test, train negs
        neg_row, neg_col = neg_adj_mask.nonzero(as_tuple=False).t()
        perm = torch.randperm(neg_row.size(0))[:row.size(0)]
        neg_row, neg_col = neg_row[perm], neg_col[perm]

        row, col = neg_row[:n_v], neg_col[:n_v]
        data.val_neg = torch.stack([row, col], dim=0)

        row, col = neg_row[n_v:n_v + n_t], neg_col[n_v:n_v + n_t]
        data.test_neg = torch.stack([row, col], dim=0)

        row, col = neg_row[n_v + n_t:], neg_col[n_v + n_t:]
        data.train_neg = torch.stack([row, col], dim=0)

    else:

        neg_adj_mask = torch.ones(data.num_nodes, data.num_nodes, dtype=torch.uint8)
        neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
        neg_adj_mask[row, col] = 0

        # Sample the test negative edges first
        neg_row, neg_col = neg_adj_mask.nonzero(as_tuple=False).t()
        perm = torch.randperm(neg_row.size(0))[:n_t]
        neg_row, neg_col = neg_row[perm], neg_col[perm]
        data.test_neg = torch.stack([neg_row, neg_col], dim=0)

        # Sample the train and val negative edges with only knowing 
        # the train positive edges
        row, col = data.train_pos
        neg_adj_mask = torch.ones(data.num_nodes, data.num_nodes, dtype=torch.uint8)
        neg_adj_mask = neg_adj_mask.triu(diagonal=1).to(torch.bool)
        neg_adj_mask[row, col] = 0

        # Sample the train and validation negative edges
        neg_row, neg_col = neg_adj_mask.nonzero(as_tuple=False).t()

        n_tot = n_v + data.train_pos.size(1)
        perm = torch.randperm(neg_row.size(0))[:n_tot]
        neg_row, neg_col = neg_row[perm], neg_col[perm]

        row, col = neg_row[:n_v], neg_col[:n_v]
        data.val_neg = torch.stack([row, col], dim=0)

        row, col = neg_row[n_v:], neg_col[n_v:]
        data.train_neg = torch.stack([row, col], dim=0)

    return data


def k_hop_subgraph(node_idx, num_hops, edge_index, max_nodes_per_hop = None,num_nodes = None):
  
    if num_nodes == None:
        num_nodes = torch.max(edge_index)+1
    row, col = edge_index
    node_mask = row.new_empty(num_nodes, dtype=torch.bool)
    edge_mask = row.new_empty(row.size(0), dtype=torch.bool)

    node_idx = node_idx.to(row.device)

    subsets = [node_idx]

    if max_nodes_per_hop == None:
        for _ in range(num_hops):
            node_mask.fill_(False)
            node_mask[subsets[-1]] = True
            torch.index_select(node_mask, 0, row, out = edge_mask)
            subsets.append(col[edge_mask])
    else:
        not_visited = row.new_empty(num_nodes, dtype=torch.bool)
        not_visited.fill_(True)
        for _ in range(num_hops):
            node_mask.fill_(False)# the source node mask in this hop
            node_mask[subsets[-1]] = True #mark the sources
            not_visited[subsets[-1]] = False # mark visited nodes
            torch.index_select(node_mask, 0, row, out = edge_mask) # indices of all neighbors
            neighbors = col[edge_mask].unique() #remove repeats
            neighbor_mask = row.new_empty(num_nodes, dtype=torch.bool) # mask of all neighbor nodes
            edge_mask_hop = row.new_empty(row.size(0), dtype=torch.bool) # selected neighbor mask in this hop
            neighbor_mask.fill_(False)
            neighbor_mask[neighbors] = True
            neighbor_mask = torch.logical_and(neighbor_mask, not_visited) # all neighbors that are not visited
            ind = torch.where(neighbor_mask==True) #indicies of all the unvisited neighbors
            if ind[0].size(0) > max_nodes_per_hop:
                perm = torch.randperm(ind[0].size(0))
                ind = ind[0][perm]
                neighbor_mask[ind[max_nodes_per_hop:]] = False # randomly select max_nodes_per_hop nodes
                torch.index_select(neighbor_mask, 0, col, out = edge_mask_hop)# find the indicies of selected nodes
                edge_mask = torch.logical_and(edge_mask,edge_mask_hop) # change edge_mask
            subsets.append(col[edge_mask])

    subset, inv = torch.cat(subsets).unique(return_inverse=True)
    inv = inv[:node_idx.numel()]

    node_mask.fill_(False)
    node_mask[subset] = True
    edge_mask = node_mask[row] & node_mask[col]

    edge_index = edge_index[:, edge_mask]

    node_idx = row.new_full((num_nodes, ), -1)
    node_idx[subset] = torch.arange(subset.size(0), device=row.device)
    edge_index = node_idx[edge_index]

    return subset, edge_index, inv, edge_mask

def plus_edge(data_observed, label, p_edge, args):
    nodes, edge_index_m, mapping, _ = k_hop_subgraph(node_idx= p_edge, num_hops=args.num_hops,\
 edge_index = data_observed.edge_index, max_nodes_per_hop=args.max_nodes_per_hop ,num_nodes=data_observed.num_nodes)
    x_sub = data_observed.x[nodes,:]
    edge_index_p = edge_index_m
    edge_index_p = torch.cat((edge_index_p, mapping.view(-1,1)),dim=1)
    edge_index_p = torch.cat((edge_index_p, mapping[[1,0]].view(-1,1)),dim=1)

    #edge_mask marks the edge under perturbation, i.e., the candidate edge for LP
    edge_mask = torch.ones(edge_index_p.size(1),dtype=torch.bool)
    edge_mask[-1] = False
    edge_mask[-2] = False

    if args.drnl == True:
        num_nodes = torch.max(edge_index_p)+1
        z = drnl_node_labeling(edge_index_m, mapping[0],mapping[1],num_nodes)
        data = Data(edge_index = edge_index_p, x = x_sub, z = z)
    else:
        data = Data(edge_index = edge_index_p, x = x_sub, z = 0)
    data.edge_mask = edge_mask

    #label = 1 if the candidate link (p_edge) is positive and label=0 otherwise
    data.label = float(label)

    return data

def minus_edge(data_observed, label, p_edge, args):
    nodes, edge_index_p, mapping,_ = k_hop_subgraph(node_idx= p_edge, num_hops=args.num_hops,\
 edge_index = data_observed.edge_index,max_nodes_per_hop=args.max_nodes_per_hop, num_nodes = data_observed.num_nodes)
    x_sub = data_observed.x[nodes,:]

    #edge_mask marks the edge under perturbation, i.e., the candidate edge for LP
    edge_mask = torch.ones(edge_index_p.size(1), dtype = torch.bool)
    ind = torch.where((edge_index_p == mapping.view(-1,1)).all(dim=0))
    edge_mask[ind[0]] = False
    ind = torch.where((edge_index_p == mapping[[1,0]].view(-1,1)).all(dim=0))
    edge_mask[ind[0]] = False
    if args.drnl == True:
        num_nodes = torch.max(edge_index_p)+1
        z = drnl_node_labeling(edge_index_p[:,edge_mask], mapping[0],mapping[1],num_nodes)
        data = Data(edge_index = edge_index_p, x= x_sub,z = z)
    else:
        data = Data(edge_index = edge_index_p, x= x_sub,z = 0)
    data.edge_mask = edge_mask

    #label = 1 if the candidate link (p_edge) is positive and label=0 otherwise
    data.label = float(label)
    return data


def load_splitted_data(args):
    par_dir = os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
    data_name=args.data_name+'_split_'+args.data_split_num
    if args.test_ratio==0.5:
        data_dir = os.path.join(par_dir, 'data/splitted_0_5/{}.mat'.format(data_name))
    else:
        data_dir = os.path.join(par_dir, 'data/splitted/{}.mat'.format(data_name))
    import scipy.io as sio
    print('Load data from: '+data_dir)
    net = sio.loadmat(data_dir)
    data = Data()

    data.train_pos = torch.from_numpy(np.int64(net['train_pos']))
    data.train_neg = torch.from_numpy(np.int64(net['train_neg']))
    data.test_pos = torch.from_numpy(np.int64(net['test_pos']))
    data.test_neg = torch.from_numpy(np.int64(net['test_neg']))

    n_pos= floor(args.val_ratio * len(data.train_pos)).int()
    nlist=np.arange(len(data.train_pos))
    np.random.shuffle(nlist)
    val_pos_list=nlist[:n_pos]
    train_pos_list=nlist[n_pos:]
    data.val_pos=data.train_pos[val_pos_list]
    data.train_pos=data.train_pos[train_pos_list]

    n_neg = floor(args.val_ratio * len(data.train_neg)).int()
    nlist=np.arange(len(data.train_neg))
    np.random.shuffle(nlist)
    val_neg_list=nlist[:n_neg]
    train_neg_list=nlist[n_neg:]
    data.val_neg=data.train_neg[val_neg_list]
    data.train_neg=data.train_neg[train_neg_list]

    data.val_pos = torch.transpose(data.val_pos,0,1)
    data.val_neg = torch.transpose(data.val_neg,0,1)
    data.train_pos = torch.transpose(data.train_pos,0,1)
    data.train_neg = torch.transpose(data.train_neg,0,1)
    data.test_pos = torch.transpose(data.test_pos,0,1)
    data.test_neg = torch.transpose(data.test_neg,0,1)
    num_nodes = max(torch.max(data.train_pos), torch.max(data.test_pos))+1
    num_nodes=max(num_nodes,torch.max(data.val_pos)+1)
    data.num_nodes = num_nodes

    return data

def load_unsplitted_data(args):
    # read .mat format files
    data_dir = os.path.join(par_dir, 'data/{}.mat'.format(args.data_name))
    print('Load data from: '+ data_dir)
    import scipy.io as sio
    net = sio.loadmat(data_dir)
    edge_index,_ = from_scipy_sparse_matrix(net['net'])
    data = Data(edge_index=edge_index)
    if is_undirected(data.edge_index) == False: #in case the dataset is directed
        data.edge_index = to_undirected(data.edge_index)
    data.num_nodes = torch.max(data.edge_index)+1
    return data
def load_Planetoid_data(args):
    print('Using data: '+ args.data_name)
    #dataset = Planetoid(root=par_dir+'/data/', name=args.data_name, transform=NormalizeFeatures())
    dataset = Planetoid(root=par_dir+'/data/', name=args.data_name)
    data = dataset[0]
    data.num_nodes = torch.max(data.edge_index)+1
    return data
# def load_Planetoid_data(args):
#     print('downloading data: '+ args.data_name)
#     #dataset = Planetoid(root=par_dir+'/data/', name=args.data_name, transform=NormalizeFeatures())
#     dataset = Planetoid(root=par_dir+'/data/', name=args.data_name)
#     # Edited from https://github.com/tkipf/gae/blob/master/gae/input_data.py
#     names = ['x', 'tx', 'allx', 'graph']
#     objects = []
#     for i in range(len(names)):
#         with open("./data/{}/raw/ind.{}.{}".format(args.data_name,args.data_name, names[i]), 'rb') as f:
#             if sys.version_info > (3, 0):
#                 objects.append(pkl.load(f, encoding='latin1'))
#             else:
#                 objects.append(pkl.load(f))
#     x, tx, allx, graph = tuple(objects)
#     filename="./data/{}/raw/ind.{}.test.index".format(args.data_name,args.data_name)
#     index = []
#     for line in open(filename):
#         index.append(int(line.strip()))
#     test_idx_reorder = index
#     test_idx_range = np.sort(test_idx_reorder)
#     if args.data_name == 'citeseer':
#         # Fix citeseer dataset (there are some isolated nodes in the graph)
#         # Find isolated nodes, add them as zero-vecs into the right position
#         test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
#         tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
#         tx_extended[test_idx_range-min(test_idx_range), :] = tx
#         tx = tx_extended
#     features = sp.vstack((allx, tx)).tolil()
#     features[test_idx_reorder, :] = features[test_idx_range, :]
#     features=torch.tensor(sp.coo_matrix.todense(features)).float()
#     adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
#     edge_index=from_scipy_sparse_matrix(adj)[0]
#     data=Data(edge_index=edge_index,x=features)
#     data.num_nodes = torch.max(data.edge_index)+1

#     return data


def set_init_attribute_representation(data,args):
    #Construct data_observed and compute its node attributes & representation
    edge_index = torch.cat((data.train_pos,data.train_pos[[1,0],:]),dim=1)
    if args.observe_val_and_injection == False:
        data_observed = Data(edge_index=edge_index)
    else:
        edge_index=torch.cat((edge_index,data.val_pos,data.val_pos[[1,0],:]),dim=1)
        data_observed = Data(edge_index=edge_index)
    data_observed.num_nodes = data.num_nodes
    if args.observe_val_and_injection == False:
        edge_index_observed = data_observed.edge_index
    else: 
        #use the injection trick and add val data in observed graph 
        edge_index_observed = torch.cat((data_observed.edge_index,\
            data.train_neg,data.train_neg[[1,0],:],data.val_neg,data.val_neg[[1,0],:]),dim=1)
    #generate the initial node attribute if there isn't any
    if data.x == None:
        if args.init_attribute =='n2v':
            from node2vec import CalN2V
            x = CalN2V(edge_index_observed,args)
        if args.init_attribute =='one_hot':
            x = F.one_hot(torch.arange(data.num_nodes), num_classes=data.num_nodes)
            x = x.float()
        if args.init_attribute =='spc':
            from SPC import spc
            x = spc(edge_index_observed,args)
            x = x.float()
        if args.init_attribute =='ones':
            x = torch.ones(data.num_nodes,args.embedding_dim)
            x = x.float()
        if args.init_attribute =='zeros':
            x = torch.zeros(data.num_nodes,args.embedding_dim)
            x = x.float()
    else:
        x = data.x
    #generate the initial node representation using unsupervised models
    if args.init_representation != None:
        val_and_test=[data.test_pos,data.test_neg,data.val_pos,data.val_neg]
        num_nodes,_=x.shape
        #add self-loop for the last node to aviod losing node if the last node dosen't have a link.
        if (num_nodes-1) in edge_index_observed:
            edge_index_observed=edge_index_observed.clone().detach()
        else:
            edge_index_observed=torch.cat((edge_index_observed.clone().detach(),torch.tensor([[num_nodes-1],[num_nodes-1]])),dim=1)
        if args.init_representation == 'gic':
            args.par_dir = os.path.abspath(os.path.join(os.path.dirname(__file__),".."))
            sys.path.append('%s/software/GIC/' % args.par_dir)
            from GICEmbs import CalGIC
            data_observed.x, auc, ap = CalGIC(edge_index_observed, x, args.data_name, val_and_test,args)

        if args.init_representation == 'vgae':
            from vgae import CalVGAE
            data_observed.x, auc, ap = CalVGAE(edge_index_observed, x, val_and_test, args)
        if args.init_representation == 'svgae':
            from svgae import CalSVGAE
            data_observed.x, auc, ap = CalSVGAE(edge_index_observed, x, val_and_test, args)
        if args.init_representation == 'argva':
            from argva import CalARGVA
            data_observed.x, auc, ap = CalARGVA(edge_index_observed, x, val_and_test, args)
        feature_results=[auc,ap]
    else:
        data_observed.x = x
        feature_results=None

    return data_observed,feature_results


def prepare_data(args):
    #load data from .mat or download from Planetoid dataset.
    
    if args.data_name in ('cora', 'citeseer', 'pubmed'):
        data = load_Planetoid_data(args)
        data = split_edges(data,args)
    else:
        if args.use_splitted == True: #use splitted train/val/test
            data = load_splitted_data(args)
        else:
            data = load_unsplitted_data(args)
            data = split_edges(data,args)
    
    

    set_random_seed(args.seed)
    data_observed,feature_results= set_init_attribute_representation(data,args)

    #Construct train, val and test data loader.
    set_random_seed(args.seed)
    train_graphs = []
    val_graphs = []
    test_graphs = []
    for i in range(data.train_pos.size(1)):
        train_graphs.append(minus_edge(data_observed,1,data.train_pos[:,i],args))

    for i in range(data.train_neg.size(1)):
        train_graphs.append(plus_edge(data_observed,0,data.train_neg[:,i],args))

    for i in range(data.test_pos.size(1)):
        test_graphs.append(plus_edge(data_observed,1,data.test_pos[:,i],args))

    for i in range(data.test_neg.size(1)):
        test_graphs.append(plus_edge(data_observed,0,data.test_neg[:,i],args))   
    if args.observe_val_and_injection == False:
        for i in range(data.val_pos.size(1)):
            val_graphs.append(plus_edge(data_observed,1,data.val_pos[:,i],args))

        for i in range(data.val_neg.size(1)):
            val_graphs.append(plus_edge(data_observed,0,data.val_neg[:,i],args))
    else:
        for i in range(data.val_pos.size(1)):
            val_graphs.append(minus_edge(data_observed,1,data.val_pos[:,i],args))

        for i in range(data.val_neg.size(1)):
            val_graphs.append(plus_edge(data_observed,0,data.val_neg[:,i],args))


    
    print('Train_link:', str(len(train_graphs)),' Val_link:',str(len(val_graphs)),' Test_link:',str(len(test_graphs)))

    train_loader = DataLoader(train_graphs,batch_size=args.batch_size,shuffle=True)
    val_loader = DataLoader(val_graphs,batch_size=args.batch_size,shuffle=True)
    test_loader = DataLoader(test_graphs,batch_size=args.batch_size,shuffle=False)

    return train_loader, val_loader, test_loader,feature_results


    
    


In [31]:
# Link prediction message passing model settings
number_of_heads_in_attention_link_encoder = 2
walk_length = 2 # Number of learning iterations based on messages passing, called "walk length" (#TODO referenciar paper)
use_mse = False
practical_neg_sample = False

# Copy validation and test ratios from experiments performed using Spektral
val_ratio = np.sum(dataset.mask_va)/number_of_nodes
test_ratio = np.sum(dataset.mask_tr)/number_of_nodes

class MyDict(dict):
    pass

args = MyDict()
args.data_name = 'cora'
args.seed = 1
args.use_splitted = False
args.practical_neg_sample = True
args.observe_val_and_injection = False
args.init_attribute=None
args.lr = 0.0005 # learning rate
args.val_ratio = 0.2
args.test_ratio = 0.2
args.init_representation = None
args.num_hops = 3
args.max_nodes_per_hop = 100
args.drnl = False
args.batch_size = 32
args.weight_decay = 0
args.epoch_num = 10


# Get device definition, by default cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instance LinkPred model
model = LinkPred(
    in_channels=number_of_node_features, # Number of nodes used as input data
    hidden_channels=number_of_labels, # Number of labels used as output data
    heads=number_of_heads_in_attention_link_encoder, # Number of attention heads
    walk_len=walk_length, 
    # drnl = args.drnl,
    # z_max = z_max, 
    MSE=use_mse
).to(device)

# Select optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
criterion = torch.nn.MSELoss(reduction='mean')

# Prompt settings to notebook
print(f'Validation ratio: {val_ratio}')
print(f'Test ratio: {test_ratio}')
print(f'Learning rate: {args.lr}')

Validation ratio: 0.18463810930576072
Test ratio: 0.051698670605613
Learning rate: 0.0005


In [32]:
# Prepare data
train_loader, val_loader, test_loader, feature_results = prepare_data(args)

Using data: cora


Processing...
Done!


Train_link: 6336  Val_link: 2110  Test_link: 2110


In [33]:
from sklearn.metrics import roc_auc_score, average_precision_score
from tqdm import tqdm

def train(loader,epoch):
    model.train()
    loss_epoch=0
    for data in tqdm(loader,desc="train"):  # Iterate in batches over the training dataset.
        data = data.to(device)
        label= data.label
        out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
        torch.cuda.empty_cache()
        loss = criterion(out.view(-1), label)  
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step()
        loss_epoch=loss_epoch+loss.item()
    return loss_epoch/len(loader)


def test(loader,data_type='test'):
    model.eval()
    scores = torch.tensor([])
    labels = torch.tensor([])
    loss_total=0
    with torch.no_grad():
        for data in tqdm(loader,desc='test:'+data_type):  # Iterate in batches over the training/test dataset.
            data = data.to(device)
            out = model(data.x, data.edge_index, data.edge_mask, data.batch, data.z)
            loss = criterion(out.view(-1), data.label)
            out = out.cpu().clone().detach()
            scores = torch.cat((scores,out),dim = 0)
            labels = torch.cat((labels,data.label.view(-1,1).cpu().clone().detach()),dim = 0)
        scores = scores.cpu().clone().detach().numpy()
        labels = labels.cpu().clone().detach().numpy()
        loss_total=loss_total+loss.item()
        return roc_auc_score(labels, scores), average_precision_score(labels, scores),loss_total

In [34]:
# Prepare TensorBoard to receive data from this training
from torch.utils.tensorboard import SummaryWriter
tb = SummaryWriter(log_dir=get_run_logdir('mpnn_walkpooling'))

# Iterate each epoch to train the model. Send performance metrics to TensorBoard
for epoch in range(1, args.epoch_num + 1):
    epoch_loss = train(train_loader, epoch)
    _, val_ap, val_loss = test(val_loader, data_type='val')
    _, test_ap, test_loss = test(test_loader, data_type='test')

    # Log performance metrics into TensorBoard
    tb.add_scalar('epoch_loss', epoch_loss, epoch) # Log epoch loss
    # tb.add_scalar('epoch_acc', val_ap, epoch) # Log epoch accuracy
    tb.add_scalar('validation_loss', val_loss, epoch) # Log epoch validation loss
    tb.add_scalar('validation_acc', val_ap, epoch) # Log epoch validation accuracy
    tb.add_scalar('test_loss', test_ap, epoch) # Log test loss
    tb.add_scalar('test_acc', test_loss, epoch) # Log test accuracy

    # Prompt to notebook
    print(f'Epoch: {epoch:03d}\t Loss: {epoch_loss:.4f}\nValidation accuracy: {val_ap:.4f}\tValidation loss: {val_loss:.4f}\nTest accuracy: {test_ap:.4f}\tTest loss: {test_loss:.4f}')


test:test: 100%|██████████| 66/66 [00:08<00:00,  7.45it/s]


Epoch: 001	 Loss: 0.2971
Validation accuracy: 0.8410	Validation loss: 0.2074
Test accuracy: 0.8268	Test loss: 0.1109


train:  20%|██        | 40/198 [00:13<00:51,  3.07it/s]


KeyboardInterrupt: 